# Utilities

Set dataset path and scale applied to spines during visualization.

In [ ]:
from notebook_widgets import SpineMeshDataset

# set dataset path and scale applied to spines during visualization
dataset_path = "0.025 0.025 0.1 dataset"
scale = (1, 1, 1)

# load and scale spine mesh dataset
spine_dataset = SpineMeshDataset().load(dataset_path)
scaled_spine_dataset = SpineMeshDataset().load(dataset_path)
scaled_spine_dataset.apply_scale(scale)

### View Dendrite Skeleton

In [ ]:
from notebook_widgets import view_skeleton_widget


display(view_skeleton_widget(scaled_spine_dataset))

### View Chords
Set `num_of_chords` and `num_of_bins`. Histograms can be exported to `dataset_path/chords_%num_of_chords%_chords_%num_of_bins%_bins.csv` file. Histograms are only exported for meshes that were viewed! Otherwise it would take too much time to calculate. Use `Calculate Metrics` cell to calculate metrics for the entire dataset.

In [ ]:
from notebook_widgets import spine_chords_widget


num_of_chords = 30000
num_of_bins = 100

display(spine_chords_widget(spine_dataset, scaled_spine_dataset, dataset_path, num_of_chords, num_of_bins))

### Calculate Metrics
Metrics for the dataset will be saved to `dataset_path/metrics.csv`. Chords histograms will be saved separately to `dataset_path/chords.csv`. 

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset


# chord method parameters
num_of_chords = 30000
num_of_bins = 100

# calculate metrics
metric_names = ["OldChordDistribution", "OpenAngle", "CVD", "AverageDistance",
                "LengthVolumeRatio", "LengthAreaRatio", "JunctionArea",
                "Length", "Area", "Volume", "ConvexHullVolume", "ConvexHullRatio",
                "LightFieldZernikeMoments"]
metric_params = [{"num_of_chords": num_of_chords, "num_of_bins": num_of_bins},
                 {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

spine_metrics = SpineMetricDataset()
spine_metrics.calculate_metrics(spine_dataset.spine_meshes, metric_names)
spine_metrics.save(f"{dataset_path}/metrics.csv")    

spine_metrics.get_metrics_subset(["OldChordDistribution"]).save_as_array(f"{dataset_path}/chords.csv")
spine_metrics.get_metrics_subset(["LightFieldZernikeMoments"]).save_as_array(f"{dataset_path}/lf_descriptor.csv")

### View Spines in Dataset

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import spine_dataset_view_widget


metrics = SpineMetricDataset().load(f"{dataset_path}/metrics.csv")
display(spine_dataset_view_widget(scaled_spine_dataset, metrics))

### Merge Manual Classifications

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from spine_analysis.spine.grouping import SpineGrouping
from notebook_widgets import create_dir
import pathlib


# load manual classifications
path = pathlib.Path(f"{dataset_path}/manual_classification")
classifications_paths = path.glob("*.json")
groupings = [SpineGrouping().load(str(path)) for path in classifications_paths]

# merge classifications
merged_grouping = SpineGrouping.merge(groupings)
merged_grouping.save(f"{dataset_path}/manual_classification/manual_classification_merged.json")

# remove spines with no consensus, filopodias and labeled-as-outliers
contested = SpineGrouping.get_contested_samples(groupings)
merged_grouping.remove_samples(contested)
merged_grouping.remove_samples(merged_grouping.groups["Filopodia"].copy())
merged_grouping.remove_samples(merged_grouping.groups["Outlier"].copy())
del merged_grouping.groups["Filopodia"]
del merged_grouping.groups["Outlier"]

merged_grouping.save(f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json")

### Consensus Table

In [ ]:
from notebook_widgets import consensus_widget
from spine_analysis.spine.grouping import SpineGrouping

# load manual classifications
path = pathlib.Path(f"{dataset_path}/manual_classification")
classifications_paths = path.glob("*.json")
groupings = [SpineGrouping().load(str(path)) for path in classifications_paths]

# show consensus table
display(consensus_widget(groupings))

### Clusters diagrams

In [ ]:
from spine_analysis.spine.grouping import SpineGrouping
import matplotlib.pyplot as plt
import pathlib
import glob

def get_distribution(classes, clusters, ax, *replasing_args) -> None:
  col = ['g','r', 'y', 'b', 'm', 'c', 'k', 'orange', 'lightgreen', 'pink', 'yellow']
  colors = {k: col[i] for i, k in enumerate(classes.group_labels)}
  #colors = {'Stubby': 'y', 'Mushroom': 'g', 'Thin': 'r'}
  print(colors)
  
  for j, lab in enumerate(clusters.group_labels):
    ss = {spike.replace(*replasing_args) for spike in clusters.groups[lab]}
    cluster_j_distribution = classes.get_spines_subset(ss.intersection([spike.replace(*replasing_args) for spikes in classes.groups.values() for spike in spikes]))
    
    labels = list(cluster_j_distribution.group_labels)
    sizes = [len(cluster_j_distribution.groups[man_lab]) for man_lab in labels]

    drawed_labels = [labels[k] for k,s in enumerate(sizes) if s > 0]
    drawed_sizes = [s for s in sizes if s > 0]
    ax[j].pie(drawed_sizes, colors = [colors[lab] for lab in drawed_labels], autopct='',
        shadow=True, startangle=90)


dataset_path = '0.025-0.025-0.1-dataset'

manual_path = pathlib.Path(f"output/clusterization/0.025-0.025-0.1-dataset_chord_kmeans_num_of_clusters=5_pca=2_5_clusters.json")
#manual_path = pathlib.Path(f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json")
classifications_paths = glob.glob("output/clusterization/zernike_abs_kernel_kmeans_num_of_clusters*.json")
groupings = [SpineGrouping().load(str(path)) for path in classifications_paths]
groupings = [groupings[4], groupings[0], groupings[-1], groupings[-3], groupings[3], groupings[2], groupings[-2], groupings[1]]
manual_grouping = SpineGrouping().load(manual_path)

fig, ax = plt.subplots(ncols=max(len(g.group_labels) for g in groupings), nrows = len(groupings), figsize=(10, 15))

for i, g in enumerate(groupings):
  get_distribution(manual_grouping, g, ax[i], '/', '\\')
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=len(manual_grouping.group_labels), nrows = len(groupings), figsize=(5, 15))

for j, lab in enumerate(manual_grouping.group_labels):
  ax[0, j].set_title(lab)
for i, g in enumerate(groupings):
  get_distribution(g, manual_grouping, ax[i], '\\', '/')
plt.show()

### cluster visualization

In [ ]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import SpineMeshDataset, clasters_spines_widget, make_viewer, widgets
from spine_segmentation import apply_scale
from spine_analysis.spine.grouping import SpineGrouping

clusterization_save_path = 'output/clusterization/zernike_real_12_kernel_kmeans_num_of_clusters=6_pca=-1_5_clusters.json'
meshes_path = '0.025 0.025 0.1 dataset'

spine_dataset = SpineMeshDataset().load(meshes_path)
spine_dataset.apply_scale((1, 1, 0.1))

clusterization = SpineGrouping()
clusterization.load(clusterization_save_path)

display(clasters_spines_widget(spine_dataset, clusterization))

### Get LF distances matrix and inspect closest/farthest spines 

In [ ]:
import numpy as np

from spine_analysis.shape_metric.approximation_metric import LightFieldZernikeMomentsSpineMetric
from notebook_widgets import SpineMeshDataset

meshes_path = '0.025 0.025 0.1 dataset'

spine_dataset = SpineMeshDataset().load(meshes_path)
lf_name = 'LightFieldZernikeMoments'

spine_metrics_lf = SpineMetricDataset()
spine_metrics_lf.calculate_metrics(spine_dataset.spine_meshes, [lf_name])
spine_metrics_lf_real = spine_metrics_lf.get_metrics_subset([lf_name])
spine_metrics_lf_real.clasterization_preprocess(zernike_postprocess='real')

meshes_names = list(spine_dataset.spine_names)
distanses = np.identity((len(meshes_names)))
distanses *= -1
for i, name_1 in enumerate(meshes_names):
  for j, name_2 in enumerate(meshes_names):
    if i == j:
      continue
    distanses[i,j] = LightFieldZernikeMomentsSpineMetric.distance(spine_metrics_lf_real.element(name_1, lf_name), spine_metrics_lf_real.element(name_2, lf_name))

# farthest spines
index = distanses.argmax()
i = index // len(meshes_names)
j = index % len(meshes_names)
print(distanses.max(), meshes_names[i], meshes_names[j])

#closest spines
distanses[distanses < 0] = 10e14
index = distanses.argmin()
i = index // len(meshes_names)
j = index % len(meshes_names)
print(distanses.min(), meshes_names[i], meshes_names[j])